In [1]:
import os

In [2]:
%pwd

'c:\\Users\\dell\\Documents\\mlops\\walmart-sales\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\dell\\Documents\\mlops\\walmart-sales'

In [ ]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/ravikiran058/walmart-sales.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "ravikiran058"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "######"

In [6]:
from dataclasses import dataclass, field
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    pipeline_path: Path 
    model_instance_path: Path
    # all_params: dict
    target_column: str
    metrics_file_name: Path
    mlflow_uri: str

In [7]:
from projectFiles.constants import *
from projectFiles.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        # params = self.params
        schema = self.schema

        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            pipeline_path = config.pipeline_path,
            model_instance_path = config.model_instance_path,
            # all_params = params,
            metrics_file_name = config.metrics_file_name,
            target_column = schema.TARGET_COLUMN.name,
            mlflow_uri = "https://dagshub.com/ravikiran058/walmart-sales.mlflow"
        )

        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import pickle

In [10]:
class ModelEvaluation:
    def __init__(self, config = ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = root_mean_squared_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, r2
    
    def log_in_mlflow(self):
        target_col = self.config.target_column
        test_df = pd.read_csv(self.config.test_data_path)
        with open(self.config.pipeline_path, 'rb') as file:
            regressor_pipeline = pickle.load(file)
        with open(self.config.model_instance_path, 'rb') as file:
            model_instance = pickle.load(file)   

        test_df.drop(columns=["Date"], inplace = True)
        test_x = test_df.drop(columns=target_col)
        test_y = test_df[target_col]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            prediction = regressor_pipeline.predict(test_x)
            
            (rmse, r2) = self.eval_metrics(test_y, prediction)

            scores = {"rmse":rmse, "r2":r2}
            save_json(path = Path(self.config.metrics_file_name), data = scores)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(regressor_pipeline, "model", registered_model_name="LGBMRegressorPipeline")
            else:
                mlflow.sklearn.log_model(regressor_pipeline, "model")

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.log_in_mlflow()
except Exception as e:
    raise e

DEBUG: YAML Content from config\config.yaml -> {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/rkdsai/walmart-sales/raw/refs/heads/main/raw_data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'data_dirs': {'features': 'artifacts/data_ingestion/features.csv', 'stores': 'artifacts/data_ingestion/stores.csv', 'train': 'artifacts/data_ingestion/train.csv', 'test': 'artifacts/data_ingestion/test.csv'}, 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_dirs': {'features': 'artifacts/data_ingestion/features.csv', 'stores': 'artifacts/data_ingestion/stores.csv', 'train': 'artifacts/data_ingestion/train.csv', 'test': 'artifacts/data_ingestion/test.csv'}}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_data_

c:\Users\dell\Documents\mlops\walmart-sales\venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[2025-03-08 23:51:28,642: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/03/08 23:51:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'LGBMRegressorPipeline'.
2025/03/08 23:52:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LGBMRegressorPipeline, version 1
Created version '1' of model 'LGBMRegressorPipeline'.


🏃 View run caring-sow-133 at: https://dagshub.com/ravikiran058/walmart-sales.mlflow/#/experiments/0/runs/83cccd790df440e2b7815a512152bf27
🧪 View experiment at: https://dagshub.com/ravikiran058/walmart-sales.mlflow/#/experiments/0
